In [1]:
import ee
import geemap
import time
import requests
import json
import os
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import geopandas as gpd
import topojson as tp

In [2]:
#ee.Authenticate()
#geemap.ee_search()
ee.Initialize()
Map = geemap.Map()

In [5]:
token='l4AhfSSNaySzhXPwpmjyY557ymEHhKbxI358fVHUXWlbSaFPaB2Zv-QLDmUpXW8io3Ofkxt2ZX_K03ob6TeLxIUp5_qxYL2-Qw8wPOVxdYmw3GHvxX_2ntfN3N5-QcmshXvfSmkW4qYgrArFcziPTfOUA8DRZE1RpYVxSJqgOL6rUq6KzBzkJ3TxRqczBLlHMoMihFBV_XnPoUFA1Qo0Zz4MJEUXX9hVt-ZaLWx1rOMHNwIwF7d57TilOQi-_ub_'
r20='https://araucaria.arauco.com/server/rest/services/Hosted/Perimetro_R20_Oficiales_VERSION_IMAGENES/FeatureServer/0/query?where=objectid+%3E%3D0&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=nom_r20&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=&having=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=xyFootprint&resultOffset=&resultRecordCount=&returnTrueCurves=false&returnCentroid=false&sqlFormat=none&resultType=&f=geojson&token='+token
###token araucaria auth (160323)
perimetro="data/r20.json"

In [6]:
df = gpd.read_file(r20)\
        .to_crs('EPSG:32718')\
        .dissolve(by='nom_r20')\
        .geometry\
        .simplify(tolerance=5)\
        .to_crs('EPSG:4326')

In [7]:
if os.path.exists(perimetro):
    os.remove(perimetro)
df.to_file(perimetro, driver="GeoJSON")

In [8]:
gdf = gpd.read_file(
    perimetro
)

In [9]:
fc = geemap.geopandas_to_ee(gdf)

In [10]:
prefire_start = '2022-07-01'
prefire_end = '2022-12-31'

# Now set the same parameters for AFTER the fire.
postfire_start = '2022-12-01'
#postfire_end = time.strftime('%Y-%m-%d', time.localtime(time.time()))
postfire_end = '2023-02-18'

In [11]:
vizfalsoColor = {
  'bands': ['B8', 'B4B', 'B3'],
  'min': 0 ,
  'max': 0.35,
  'gamma': [1, 1, 1]
}

vizfalsoColor2 = {
  'bands': ['B8', 'B4', 'B3'],
  'min': 0 ,
  'max': 0.35,
  'gamma': [1, 1, 1]
}

vizfire = {
  'bands': ['B11', 'B8B', 'B4'],
  'min': 0.17 ,
  'max': 0.35,
  'gamma': [1, 1, 1]
}

In [12]:
countries = ee.FeatureCollection('users/raulperezastorga/Limites_SPPP')
roi = countries

In [13]:
def addNBR(image):
    nbr = image.normalizedDifference(['B8', 'B12']).rename("nbr")
    image = image.addBands(nbr)
    return image

In [14]:
def red2(image):
    red= image.expression('(RED*1.3)',{'RED': image.select('B4') }).rename('B4B')
    image = image.addBands(red)
    return image

In [16]:
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked image, scaled to [0, 1].
  return image.updateMask(mask).divide(10000)

In [17]:
# Define a function to simplify the geometries in the feature collection
def simplifyGeom(feature):
  return feature.simplify(tolerance=50)

In [18]:
# Define function to dissolve features based on an attribute name shared across all features (e.g., 'name')
def dissolve_by_attribute(fc, attribute):
    # Create a distinct list of attribute values for the given attribute name
    attributes = fc.distinct(attribute).aggregate_array(attribute)
    # Dissolve the polygons by each attribute value
    dissolved_fc = ee.FeatureCollection(attributes.map(
        lambda attr: ee.Feature(None, {
            attribute: ee.String(attr),
            'geometry': ee.Geometry.MultiPolygon(
                fc.filter(ee.Filter.eq(attribute, attr)).geometry().dissolve())
        })))
    return dissolved_fc


In [19]:
prefireImCol = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                  .filterDate(prefire_start, prefire_end) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                  .sort('CLOUDY_PIXEL_PERCENTAGE')\
                  .filterBounds(roi) \
                  .sort('system:time_start')\
                  .map(maskS2clouds)\
                  .map(addNBR)
                  
postfireImCol = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                  .filterDate(postfire_start, postfire_end) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1)) \
                  .filterBounds(roi) \
                  .sort('system:time_start')\
                  .map(maskS2clouds)\
                  .map(addNBR)\
                  .map(red2)

#HARMONIZE
prefire = prefireImCol.max().clip(roi)
postfire = postfireImCol.min().clip(roi)

In [20]:
# firms = ee.ImageCollection('FIRMS')
# fires_firms = ee.ImageCollection(firms\
#         .select('T21')\
#         .filterDate(prefire_start,postfire_end)\
#         .filterBounds(roi)) 
# fires_firmsmx = fires_firms.max().clip(roi).subtract(273.15)
# fires_firmsrc = fires_firmsmx.gte(100)
# fires_firmsvc= fires_firmsrc.reduceToVectors(**{
#   'geometry':roi,
#   'crs': 'EPSG:32718',
#   'scale':500,
#   'geometryType':'polygon',
#   'eightConnected': False,
#   'bestEffort':False,
#   'maxPixels':9e10,
# }
#   )

# r20=ee.FeatureCollection(fires_firmsvc.toList(fires_firmsvc.size()))\
#     .filterMetadata('label', 'greater_than', 0)\
#     .filterMetadata('count', 'greater_than', 10)\
#     .map(lambda f: f.buffer(500))\
#     .union()\
#     .geometry()\
#     .simplify(50)


r20 = fc\
    .map(lambda f: f.buffer(500))\
    .union()

In [21]:
preNBR = prefire.select('nbr')
postNBR = postfire.select('nbr')
dNBR_withoutsmooth = preNBR.subtract(postNBR)
dNBR_unscaled = dNBR_withoutsmooth.reduceNeighborhood(
    reducer=ee.Reducer.mean(),
    kernel=ee.Kernel.gaussian(radius=5),
)
#dNBR_unscaled=dNBR_unscaled.clip(r20)
dNBR_reclass = dNBR_unscaled.gte(-0.5).add(dNBR_unscaled.gte(-0.25)).add(dNBR_unscaled.gte(-0.1)).add(dNBR_unscaled.gte(0.1)).add(dNBR_unscaled.gte(0.27)).add(dNBR_unscaled.gte(0.44)).add(dNBR_unscaled.gte(0.66))
#dNBR_Seguro = dNBR_unscaled.gte(0.27)

viz_dNBR={
  'min':1,
  'max':7,#"#7a8737","#acbe4d","#0ae042",
  'palette':["#7a8737","#acbe4d","#0ae042","#fff70b","#ffaf38","#ff641b","#a41fd6"]
}

In [22]:
clip_fc= dNBR_reclass.clip(r20)

In [23]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [24]:
Map.addLayer(postfire,vizfalsoColor,"FalsoColor post")
#Map.addLayer(dNBR_reclass, viz_dNBR, 'dNBR Clase')
#Map.addLayer(fc, {}, "geopandas to ee")
#Map.addLayer(comp,vizfire,"FalsoColor fire")
#Map.layers

In [25]:
Map.addLayer(prefire,vizfalsoColor2,"FalsoColor pre")

In [26]:
Map.addLayer(dNBR_reclass, viz_dNBR, 'dNBR Clase')

In [27]:
Map.addLayer(clip_fc, viz_dNBR, 'dNBR Clase')

In [28]:
Map.addLayer(fc, {}, "geopandas to ee")

In [29]:
Map.layers

(TileLayer(attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', base=True, max_zoom=19, min_zoom=1, name='OpenStreetMap.Mapnik', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://tile.openstreetmap.org/{z}/{x}/{y}.png'),
 TileLayer(attribution='Google', max_zoom=22, name='Google Maps', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}'),
 TileLayer(attribution='Google Earth Engine', max_zoom=24, name='FalsoColor post', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/ma

In [107]:
dNBR_vector= dNBR_reclass.reduceToVectors(**{
'geometry':r20,
'crs': 'EPSG:32718',
'scale':30,
'geometryType':'polygon',
'eightConnected': False,
'bestEffort':False,
'maxPixels':9e10,
'labelProperty': 'zone',
}
)
dNDBR_export=ee.FeatureCollection(dNBR_vector.toList(dNBR_vector.size()))
#Dissolve the simplified polygons and reduce file size
dissolved_polygon_fc = dissolve_by_attribute(dNDBR_export, 'zone')

    #.filterMetadata('label', 'greater_than', 3)
geemap.ee_export_vector_to_drive(
    dNDBR_export, description=postfire_end+"_dNBR_mxmin_500m", fileFormat='GeoJSON', folder="dNBR"
)



Exporting 2023-03-23_dNBR_mxmin_500m... Please check the Task Manager from the JavaScript Code Editor.


In [99]:
# dNBR_vector= dNBR_reclass.reduceToVectors(**{
# 'geometry':r20,
# 'crs': 'EPSG:32718',
# 'scale':20,
# 'geometryType':'polygon',
# 'eightConnected': False,
# 'bestEffort':False,
# 'maxPixels':9e10,
# }
# )
# dNDBR_export=ee.FeatureCollection(dNBR_vector.toList(dNBR_vector.size()))\
#     .map(simplifyGeom)
# # Dissolve the simplified polygons and reduce file size
# dissolved_polygon_fc = dissolve_by_attribute(dNDBR_export, 'label')

#     #.filterMetadata('label', 'greater_than', 3)
# geemap.ee_export_vector_to_drive(
#     dissolved_polygon_fc, description=postfire_end+"_dNBR_500m", fileFormat='GeoJSON', folder="dNBR"
# )



In [100]:
# def export_drive(i):
#   zona='{}'.format(i)
#   area = ee.FeatureCollection('projects/gl-for-gis-dsr/assets/OrganizationalFeatures/Limite_SPPP_1')\
#       .filter(ee.Filter.equals("idzona", int(zona)))
#   dNBR_vector= dNBR_reclass.reduceToVectors(**{
#     'geometry':area,
#     'crs': 'EPSG:32718',
#     'scale':30,
#     'geometryType':'polygon',
#     'eightConnected': False,
#     'bestEffort':False,
#     'maxPixels':9e10,
#   }
#     )
#   dNDBR_export=ee.FeatureCollection(dNBR_vector.toList(dNBR_vector.size()))
#   geemap.ee_export_vector_to_drive(
#       dNBR_vector, description=postfire_end+"_dNBR_" + zona, fileFormat='GeoJSON', folder="dNBR"
#  )

In [101]:
# export_drive(4)

In [102]:
# for i in zona:
#     export_drive(i)

In [103]:
# dNBR_vector= dNBR_reclass.reduceToVectors(**{
# 'geometry':r20,
# 'crs': 'EPSG:32718',
# 'scale':30,
# 'geometryType':'polygon',
# 'eightConnected': False,
# 'bestEffort':False,
# 'maxPixels':9e10,
# }
# )
# dNDBR_export=ee.FeatureCollection(dNBR_vector.toList(dNBR_vector.size()))\
#     .filterMetadata('label', 'greater_than', 3)
# geemap.ee_export_vector_to_cloud_storage(
#     dNDBR_export, description=postfire_end+"_dNBR", fileFormat='csv',bucket='ee_geomatica'
# )

# def export_bucket(i):
#     sppp='{}'.format(i)
#     area = roi\
#         .filter(ee.Filter.equals("PaisajeID", int(sppp)))
#     dNBR_vector= dNBR_reclass.reduceToVectors(**{
#     'geometry':area,
#     'crs': 'EPSG:32718',
#     'scale':20,
#     'geometryType':'polygon',
#     'eightConnected': False,
#     'bestEffort':False,
#     'maxPixels':9e10,
#     }
#     )
#     dNDBR_export=ee.FeatureCollection(dNBR_vector.toList(dNBR_vector.size()))
#     geemap.ee_export_vector_to_cloud_storage(
#         dNBR_vector, description=postfire_end+"_dNBR_" + str(sppp), fileFormat='SHP',bucket='bck-gl-for-gis-dsr-tfstate/EE_prueba/02_dNBR_vector'
#     )

In [104]:
# ppp=list(range(21,23))
# for i in ppp:
#     export_drive(i)

In [105]:
#geemap.ee_search()